# OpenSpiel + Gambit Workflow on one card poker

In this tutorial, we will:

1. Load examples of normal form and extensive form games in OpenSpiel and Gambit
2. Train agents in OpenSpiel to play games and create strategies
3. Compare results against equilibria computed with Gambit

This notebook demonstrates the workflow between OpenSpiel and Gambit for game-theoretic analysis:

- **OpenSpiel**: Provides iterative learning algorithms for strategy approximation
- **Gambit**: Provides exact equilibrium computation for theoretical comparison

In [1]:
import pygambit as gbt
import pyspiel
from open_spiel.python.egt.utils import game_payoffs_array
from open_spiel.python.algorithms.gambit import export_gambit
from open_spiel.python.egt import dynamics
import numpy as np

## Strategic form example: Rock-Paper-Scissors

Load matrix rock-paper-scissors from OpenSpiel:

In [2]:
ops_matrix_rps_game = pyspiel.load_game("matrix_rps")

Get the payoffs as numpy arrays...

In [3]:
matrix_rps_payoffs = game_payoffs_array(ops_matrix_rps_game)
matrix_rps_payoffs

array([[[ 0., -1.,  1.],
        [ 1.,  0., -1.],
        [-1.,  1.,  0.]],

       [[ 0.,  1., -1.],
        [-1.,  0.,  1.],
        [ 1., -1.,  0.]]])

... which we can use to recreate the game in Gambit:

In [4]:
gbt_matrix_rps_game = gbt.Game.from_arrays(
    matrix_rps_payoffs[0],  # Player 1 payoffs
    matrix_rps_payoffs[1],  # Player 2 payoffs
    title="Matrix Rock-Paper-Scissors"
)

# Add labels to the strategies
for player in gbt_matrix_rps_game.players:
    player.strategies[0].label = "Rock"
    player.strategies[1].label = "Paper"
    player.strategies[2].label = "Scissors"

gbt_matrix_rps_game

,Rock,Paper,Scissors
Rock,"0.0,0.0","-1.0,1.0","1.0,-1.0"
Paper,"1.0,-1.0","0.0,0.0","-1.0,1.0"
Scissors,"-1.0,1.0","1.0,-1.0","0.0,0.0"


The equilibrium strategy for both players is to choose rock, paper, and scissors with equal probability:

In [5]:
gbt.nash.lcp_solve(gbt_matrix_rps_game).equilibria[0]

[[Rational(1, 3), Rational(1, 3), Rational(1, 3)], [Rational(1, 3), Rational(1, 3), Rational(1, 3)]]

We can use OpenSpiel's dynamics module to demonstrate evolutionary game theory dynamics, or "replicator dynamics", which models how strategy population frequencies change over time based on relative fitness/payoffs.

Let's start with an initial population that is not at equilibrium, but weighted quite heavily towards scissors with proportions: 20% Rock, 20% Paper, 60% Scissors:

In [6]:
dyn = dynamics.SinglePopulationDynamics(matrix_rps_payoffs, dynamics.replicator)
x = np.array([0.2, 0.2, 0.6])
dyn(x)

array([ 0.08, -0.08,  0.  ])

`dyn(x)` calculates the rate of change (derivative) for each strategy in the current population state and returns how fast each strategy's frequency is changing.

In replicator dynamics, strategies that perform better than average will increase in frequency, while strategies performing worse will decrease. Since Scissors beats Paper but loses to Rock, and this population has few Rock players, we'd expect:

- Scissors frequency might decrease (vulnerable to Rock)
- Rock frequency might increase (beats the abundant Scissors)
- Paper frequency might decrease (loses to abundant Scissors)

This is part of the evolutionary path toward the Nash equilibrium where all three strategies have equal frequency (1/3 each) in Rock-Paper-Scissors.

In [7]:
x = np.array([0.25, 0.25, 0.5])
alpha = 0.01
for i in range(10000):
    x += alpha * dyn(x)
print(x)

[0.17411743 0.45787641 0.36800616]


<!-- ## Extensive form example: Silly1111 Poker -->

<!-- Silly poker is a variant imperfect information one-card poker game introduced in tutorial 3, but in which there are 3 possible cards (J, Q, K) instead of 2. -->

## Extensive form example: Tiny Hanabi

For extensive form games, OpenSpiel can export to the EFG format used by Gambit. Here we demonstrate this with Tiny Hanabi, loaded from the OpenSpiel [game library](https://openspiel.readthedocs.io/en/latest/games.html).


Note: as of OpenSpiel `1.6.1`, many of the games in the game library do not produce correct EFG exports. For example, Kuhn Poker EFG export did not produce a valid `.efg` file for Gambit, giving the error:

```
ValueError: Parse error in game file: Probabilities must sum to exactly one
```

In [8]:
ops_hanabi_game = pyspiel.load_game("tiny_hanabi")
efg_hanabi_game = export_gambit(ops_hanabi_game)
efg_hanabi_game

'EFG 2 R "tiny_hanabi()" { "Pl0" "Pl1" } \nc "" 1 "" { "d0" 0.5000000000000000 "d1" 0.5000000000000000  } 0\n c "p0:d0" 2 "" { "d0" 0.5000000000000000 "d1" 0.5000000000000000  } 0\n  p "" 1 1 "" { "p0a0" "p0a1" "p0a2"  } 0\n   p "" 2 1 "" { "p1a0" "p1a1" "p1a2"  } 0\n    t "" 1 "" { 10.0 10.0 }\n    t "" 2 "" { 0.0 0.0 }\n    t "" 3 "" { 0.0 0.0 }\n   p "" 2 2 "" { "p1a0" "p1a1" "p1a2"  } 0\n    t "" 4 "" { 4.0 4.0 }\n    t "" 5 "" { 8.0 8.0 }\n    t "" 6 "" { 4.0 4.0 }\n   p "" 2 3 "" { "p1a0" "p1a1" "p1a2"  } 0\n    t "" 7 "" { 10.0 10.0 }\n    t "" 8 "" { 0.0 0.0 }\n    t "" 9 "" { 0.0 0.0 }\n  p "" 1 1 "" { "p0a0" "p0a1" "p0a2"  } 0\n   p "" 2 4 "" { "p1a0" "p1a1" "p1a2"  } 0\n    t "" 10 "" { 0.0 0.0 }\n    t "" 11 "" { 0.0 0.0 }\n    t "" 12 "" { 10.0 10.0 }\n   p "" 2 5 "" { "p1a0" "p1a1" "p1a2"  } 0\n    t "" 13 "" { 4.0 4.0 }\n    t "" 14 "" { 8.0 8.0 }\n    t "" 15 "" { 4.0 4.0 }\n   p "" 2 6 "" { "p1a0" "p1a1" "p1a2"  } 0\n    t "" 16 "" { 0.0 0.0 }\n    t "" 17 "" { 0.0 0.0

Now let's load the EFG in Gambit:

In [9]:
with open("games/hanabi.efg", "w") as f:
    f.write(efg_hanabi_game)
gbt_hanabi_game = gbt.read_efg("games/hanabi.efg")
gbt_hanabi_game

Game(title='tiny_hanabi()')

In [10]:
gbt.nash.lcp_solve(gbt_hanabi_game).equilibria[0]

[[[Rational(0, 1), Rational(0, 1), Rational(1, 1)], [Rational(0, 1), Rational(1, 1), Rational(0, 1)]], [[Rational(0, 1), Rational(0, 1), Rational(1, 1)], [Rational(0, 1), Rational(1, 1), Rational(0, 1)], [Rational(1, 1), Rational(0, 1), Rational(0, 1)], [Rational(0, 1), Rational(0, 1), Rational(1, 1)], [Rational(0, 1), Rational(1, 1), Rational(0, 1)], [Rational(0, 1), Rational(0, 1), Rational(1, 1)]]]